In [1]:
import numpy as np
import math
import pandas as pd
from utils import *
import torch
from skmultilearn.problem_transform import BinaryRelevance
from sklearn.base import BaseEstimator
from torch import nn


In [2]:
class FFN(nn.Module):
    def __init__(self, INPUT_DIM, HIDDEN_DIM, DEVICE='cuda'):
        super(FFN, self).__init__()
        self.NAME = "FFN"
        self.HIDDEN_DIM = HIDDEN_DIM
        self.INPUT_DIM = INPUT_DIM
        self.devivce = DEVICE


        self.input = nn.Linear(self.INPUT_DIM, self.HIDDEN_DIM).to(self.devivce)
        self.out = nn.Linear(self.HIDDEN_DIM, 1)



    def forward(self, X):
        X1 = torch.sigmoid(self.input(X))
        X2 = self.out(X1)

        y_hat = X2.view(-1)

        return y_hat

    def computeLoss(self, logits, labels):
        criterion = nn.BCEWithLogitsLoss()
        loss = criterion(logits, labels)
        return loss

In [3]:
####### FFN 

class FfnClf(BaseEstimator): # Inherits scikit-learn base classifier
    '''Let's your model be used by LearningWithNoisyLabels'''
    def __init__(self, input_size, hidden_size, num_classes, batch_size, device='cuda', learning_rate = 1e-2, num_epochs = 200, verbose=True):

        self.device = device
        self.model = FFN(input_size, hidden_size).to(self.device)

        self.learning_rate = learning_rate
        self.num_epochs = num_epochs
        
        self.batch_size = batch_size
        self.verbose = verbose


    def fit(self, X, y, sample_weight = None):

        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=1e-3) #weight_decay=1e-3
        lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(self.optimizer, gamma=0.99)

        train_dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32), 
                                                       torch.tensor(y, dtype=torch.float32))
        train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)


        # --- train the model ---
        total_step = len(train_loader)
        training_hidden_list = []
        training_images = []
        loss_list = []
        for epoch in range(self.num_epochs):
            final_loss = 0.0
            for i, (X, y) in enumerate(train_loader):
                X = X.to(self.device)

                y = y.to(self.device)

                y = y.float()

                # --- Forward pass ---
                y_logit = self.model(X)
                
                # --- Backward and optimize ---
                self.optimizer.zero_grad()
                loss = self.model.computeLoss(y_logit, y.float())
                final_loss += loss.item()
                loss.backward()
                self.optimizer.step()
                
                if self.verbose:
                    if (i+1) % 1 == 0:
                        print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' .format(epoch+1, self.num_epochs, i+1, total_step, loss.item()))
            loss_list.append(final_loss/float(i))
            lr_scheduler.step()

    def predict(self, X):
        y = copy.copy(X)
        test_dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32), 
                                                       torch.tensor(y, dtype=torch.float32))
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

        with torch.no_grad():
            data = []
            preds = []
            for X, y in test_loader:
                X = X.to(self.device)

                y_logit = self.model(X)
                y_pred_prob = torch.sigmoid(y_logit)

                y_pred = np.round(y_pred_prob.cpu())
                preds.append(y_pred)
            preds = torch.cat(preds).numpy()

        return preds


    def predict_proba(self, X):
        y = copy.copy(X)
        test_dataset = torch.utils.data.TensorDataset(torch.tensor(X, dtype=torch.float32), 
                                                       torch.tensor(y, dtype=torch.float32))
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False)

        with torch.no_grad():
            data = []
            preds = []
            for X, y in test_loader:
                X = X.to(self.device)

                y_logit = self.model(X)
                y_pred_prob = torch.sigmoid(y_logit)
                y_pred = y_pred_prob.cpu()
                preds.append(y_pred)
            preds = torch.cat(preds).numpy()
            preds = np.hstack(((1. - preds).reshape(-1,1),preds.reshape(-1,1)))
            print(preds.shape)

        return preds

In [4]:
classes = ['Beach','Sunset','FallFoliage','Field','Mountain','Urban']

input_features = ['Att' + str(i) for i in range(1,295)]

In [5]:
exp_name = 'scene'

In [6]:
train_data, train_labels = load_scene(partition='Train')
train_labels = make_binary(train_labels, classes)
for c in train_labels.columns:
    print("'" + c +"',")

'Beach',
'Sunset',
'FallFoliage',
'Field',
'Mountain',
'Urban',


In [7]:
# --- Device configuration ---
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# --- Hyper-parameters ---
input_size = len(input_features)
hidden_size = 100
n_hidden = 1
num_classes = len(classes)
batch_size = 128
num_epochs = 200
learning_rate = 1e-3

train_df = train_data[input_features]
train_ls = train_labels[classes]

cutoff = math.floor(train_df.shape[0]/2.)
first_half_data = train_df[:cutoff]
last_half_data = train_df[cutoff:]
first_half_ls = train_ls[:cutoff]
last_half_ls = train_ls[cutoff:]

clf = BinaryRelevance(classifier=FfnClf(input_size, hidden_size, num_classes, batch_size, device, learning_rate, num_epochs, verbose=False))
clf.fit(first_half_data, first_half_ls)

preds_last_half = clf.predict(last_half_data.values)


clf = BinaryRelevance(classifier=FfnClf(input_size, hidden_size, num_classes, batch_size, device, learning_rate, num_epochs, verbose=False))
clf.fit(last_half_data, last_half_ls)

preds_first_half = clf.predict(first_half_data.values)

preds_first = np.asarray(preds_first_half.todense())
preds_last = np.asarray(preds_last_half.todense())
preds_combined = np.vstack((preds_first, preds_last))
errors = train_ls.values - preds_combined
error_df = pd.DataFrame(errors, columns = classes)

In [9]:
error_df.to_pickle('error_df.pkl')